# module

In [2]:
def get_dendrogram_segments(icoord, dcoord):
    """Linkage Matrixから描画用セグメントを生成。簡易実装。"""
  
    segments = []
    for icoords, dcoords in zip(icoord, dcoord):
        x1, x2, x3, x4 = icoords
        y1, y2, y3, y4 = dcoords
        segments.append([(x1, y1), (x2, y2)])
        segments.append([(x2, y2), (x3, y3)])
        segments.append([(x4, y4), (x3, y3)])
    return segments

In [4]:
import plotly.graph_objects as go
import plotly.graph_objects as go
HIGHLIGHT_COLORS = {
    'default': 'skyblue',
    'dr_selection': 'orange',
    'heatmap_click': 'red',
}
def plot_dendrogram_plotly(segments, 
                           colors=None, 
                           scores=None, 
                           is_selecteds=None,
                           is_heatmap_clicked=None,
                           **kwargs):
    """Plotlyを使用したデンドログラムの描画（segmentsは線分タプルのリスト）"""

    # kwargsに含まれる情報が有効かチェック
    additional_data = []
    
    # kwargs内のリストも展開して保存
    for key, value_list in kwargs.items():
        
        # ✅ 有効なリストは3倍に展開してadditional_dataに追加
    
        additional_data.append((key, value_list))
        print(f"length of expanded {key}: {len(value_list)}")
    
  
    fig = go.Figure()
    for i, seg in enumerate(segments):
        index = i // 3  # 3つのセグメントごとに1つのクラスタに対応
       
        x_coords = [seg[0][0], seg[1][0]]
        y_coords = [seg[0][1], seg[1][1]]
        # 色設定の優先順位: heatmapクリック > DR選択 > デフォルト
        opacity = 1.0
        
        # heatmapクリックが最優先
        if is_heatmap_clicked is not None and is_heatmap_clicked[index]:
            color = HIGHLIGHT_COLORS['heatmap_click']
        elif is_selecteds is not None and is_selecteds[index]:
            color = HIGHLIGHT_COLORS['dr_selection']
        else:
            color = HIGHLIGHT_COLORS['default']

        hover_lines = []
        for key, value_list in additional_data:
            value = value_list[index] if index < len(value_list) else "N/A"
            if isinstance(value, float):
                hover_lines.append(f"{key}: {value:.4f}")
            else:
                hover_lines.append(f"{key}: {value}")
        
        full_hover_text = '<br>'.join(hover_lines)
        
        fig.add_trace(go.Scatter(
            x=x_coords,
            y=y_coords,
            mode='lines',
            line=dict(color=color, width=1),
            showlegend=False,
            hoverinfo='text' if (is_selecteds is None or is_selecteds[index]) else 'skip',
            text=[full_hover_text] * len(x_coords),
            opacity=opacity
        ))

    return fig

In [5]:
def _get_leaves(condensed_tree):
    cluster_tree = condensed_tree[condensed_tree['child_size'] > 1]
    print(len(cluster_tree))
    if cluster_tree.shape[0] == 0:
        # Return the only cluster, the root
        return [condensed_tree['parent'].min()]

    root = cluster_tree['parent'].min()
    return _recurse_leaf_dfs(cluster_tree, root)
  
def _recurse_leaf_dfs(cluster_tree, current_node):
  children = cluster_tree[cluster_tree['parent'] == current_node]['child']
  if len(children) == 0:
      return [current_node,]
  else:
      return sum([_recurse_leaf_dfs(cluster_tree, child) for child in children], [])
  
def get_leaves(cluster_tree):
    """
    cluster_tree: (u, v, lambda_val, child_size, parent)
    """
    root = cluster_tree[:, 2].max()
    print(f"root: {root}")
    return recurse_leaf_dfs(cluster_tree, root)
    

def recurse_leaf_dfs(cluster_tree, current_node):
    # print(f"Visiting Node: {current_node}")
    child1 = cluster_tree[cluster_tree[:,2] == current_node][:,0]
    child2 = cluster_tree[cluster_tree[:,2] == current_node][:,1]
    # print(f"Children of Node {current_node}: Child1 {child1}, Child2 {child2}")

    if len(child1) == 0 and len(child2) == 0:
        
        return [current_node,]
    else:
        return sum([recurse_leaf_dfs(cluster_tree, child) for child in np.concatenate((child1, child2))], [])


def get_linkage_matrix_from_hdbscan(condensed_tree):
    """
    (child1, child2, parent, lambda_val, count)
    """
    print("Generating linkage matrix from HDBSCAN condensed tree...")
    linkage_matrix = []
    raw_tree = condensed_tree._raw_tree
    condensed_tree = condensed_tree.to_pandas()
    cluster_tree = condensed_tree[condensed_tree['child_size'] > 1]
    sorted_condensed_tree = cluster_tree.sort_values(by=['lambda_val','parent'], ascending=True)
    print(f"len of sorted condensed tree: {len(sorted_condensed_tree)}")

    for i in range(0, len(sorted_condensed_tree), 2):
    
        # 偶数行（i）と次の奇数行（i+1）をペアとして取得
        if i + 1 < len(sorted_condensed_tree):
            
            row_a = sorted_condensed_tree.iloc[i]
            row_b = sorted_condensed_tree.iloc[i+1]
            
            # **前提チェック**: lambda_valが同じであることを確認
            if row_a['lambda_val'] != row_b['lambda_val']:
                # lambda_valが異なる場合は、次の処理に進む（結合の前提が崩れる）
                raise ValueError(f"Lambda value mismatch at rows {i} and {i+1}: {row_a['lambda_val']} vs {row_b['lambda_val']}")
                
            # Parent IDが同じであることを確認 (同じ結合の結果である可能性が高い)
            if row_a['parent'] != row_b['parent']:
                # Parent IDが異なる場合は、このペアは単一の結合ではない可能性が高い
                raise ValueError(f"Parent ID mismatch at rows {i} and {i+1}: {row_a['parent']} vs {row_b['parent']}")
            
            child_a = row_a['child']
            child_b = row_b['child']
            lam = row_a['lambda_val']
            
            # count (サイズ) は、結合された2つの子ノードのサイズ合計を使うのが論理的だが、
            # HDBSCANは親ノードのサイズをリストで持っているため、ここではそのサイズを使用
            # より正確には、このParent IDを持つ全子ノードのサイズの合計を使うべきだが、
            # 2行の child_size の合計で暫定的に対応
            # total_size = row_a['child_size'] + row_b['child_size']


            total_size = raw_tree[raw_tree['child'] == row_a['parent']]['child_size']
            if len(total_size) == 0:
                total_size = row_a['child_size'] + row_b['child_size']
            else:
                total_size = total_size[0]
            # print(total_size)
            parent_id = row_a['parent']

            linkage_matrix.append([
                int(child_a), 
                int(child_b), 
                int(parent_id),
                lam, 
                total_size,
        ])   
    print(f"len of linkage matrix: {len(linkage_matrix)}")


    # 葉ノードに0-N-1のIDを振る
    node_id_map = {}
    current_id = 0
    leaves = _get_leaves(raw_tree)
    print(f"Number of leaves: {len(leaves)}")

    for leaf in leaves:
        node_id_map[int(leaf)] = current_id
        current_id += 1

    print(f"Leaf ID Map Size: {len(node_id_map)}")
    print(f"current id: {current_id}")

    # 結合ノードにIDを振る(linkage matrixのparent)
    for row in linkage_matrix.__reversed__():
        parent_id = row[2]
        if parent_id not in node_id_map:
            node_id_map[parent_id] = current_id
            current_id += 1

        else:
            print(f"Duplicate Parent ID found: {parent_id}")
            raise ValueError(f"Node ID {parent_id} already assigned!")
    print(f"Total Node ID Map Size: {len(node_id_map)}")
    print(f"current_id: {current_id}")

    # linkage matrixを書き換え
    max_lambda = max(row[3] for row in linkage_matrix)
    print(f"Max Lambda: {max_lambda}")
    linkage_matrix_mapped = [ 
        [node_id_map[row[0]], node_id_map[row[1]], node_id_map[row[2]], max_lambda - row[3], row[4]] 
        for row in linkage_matrix.__reversed__()
    ]

    return np.array(linkage_matrix_mapped), node_id_map # linkage matrix, parentid -> newid



# Data

In [7]:
import pickle
import numpy as np


with open('../18_rapids/result/20251203_053328/condensed_tree_object.pkl', 'rb') as f:
    condensed_tree = pickle.load(f)

with open('../19_tree/processed_data/cluster_similarities.pkl', 'rb') as f:
    cluster_similarities = pickle.load(f)

print(cluster_similarities.keys())
linkage_matrix, cluster_to_id_map = get_linkage_matrix_from_hdbscan(condensed_tree)
n_clusters = linkage_matrix.shape[0] + 1
print(f"cluster n : {n_clusters}")
cluster_similarity_dict = cluster_similarities["mahalanobis_distance"]


# reverse
id_to_cluster_map = {k: v for v, k in cluster_to_id_map.items()}
print(f"head map:{list(id_to_cluster_map.items())[:5]}")

# make cluster similarity matrix
# cluster id, cluster id -> linkage matrix id

cluster_similarity = np.zeros((n_clusters, n_clusters))

import numpy as np

# --- 前提 ---
# n_clusters: リーフノードの総数 N
# cluster_similarity: N x N の numpy.zeros 行列
# cluster_similarity_dict: {(l1, l2): distance} 形式の辞書
# id_to_cluster_map: {cluster_index (0~N-1): cluster_id (l1, l2で使用)} 形式の辞書
# ------------------

# 対角成分の処理を含めるため、ループ後に np.fill_diagonal を実行する方が簡潔です。
# ただし、ここではご要望通り、ループ内で値を設定します。
for cluster1 in range(n_clusters):
    for cluster2 in range(n_clusters):
        
        # 自身との距離（対角成分）は必ず0に設定
        if cluster1 == cluster2:
            cluster_similarity[cluster1, cluster2] = 0.0
            continue # 次のループへ

        l1 = id_to_cluster_map.get(cluster1)
        l2 = id_to_cluster_map.get(cluster2)
        
        # 1. 最初に (l1, l2) の順で値を取得を試みる
        similarity = cluster_similarity_dict.get((l1, l2))
        
        # 2. 値が None（キーが存在しない）の場合、キーを反転して (l2, l1) の順で検索
        if similarity is None:
             # 反転キー (l2, l1) で検索。それでも見つからなければ 0.0 とする。
             similarity = cluster_similarity_dict.get((l2, l1), 0.0)

        # 3. 行列への代入と対称性の確保
        
        # 行列の (cluster1, cluster2) に値を代入
        cluster_similarity[cluster1, cluster2] = similarity
        
        # 対称性の確保: (cluster2, cluster1) にも同じ値を代入
        # これにより、行列全体が対称になります。
        cluster_similarity[cluster2, cluster1] = similarity


print(f"head similarity: {cluster_similarity[:5, :5]}")


dict_keys(['kl_divergence', 'bhattacharyya_coefficient', 'mahalanobis_distance'])
Generating linkage matrix from HDBSCAN condensed tree...
len of sorted condensed tree: 884
len of linkage matrix: 442
884
Number of leaves: 443
Leaf ID Map Size: 443
current id: 443
Total Node ID Map Size: 885
current_id: 885
Max Lambda: 2.4746105670928955
cluster n : 443
head map:[(0, 115760), (1, 115763), (2, 115769), (3, 115771), (4, 115774)]
head similarity: [[  0.          56.91189655  58.49257851 135.87115276  30.79106656]
 [ 56.91189655   0.          11.79247147  10.00750598   7.55256419]
 [ 58.49257851  11.79247147   0.          16.13170821   4.30812969]
 [135.87115276  10.00750598  16.13170821   0.           7.3173488 ]
 [ 30.79106656   7.55256419   4.30812969   7.3173488    0.        ]]


# ソートアルゴリズムの定義

In [8]:
def compute_dendrogram_coords_no_sort(Z, n_points):
    """
    Linkage Matrixからデンドログラム描画用の座標を計算します。
    枝の回転（ソート）は一切行いません。Z行列の結合順序通りに配置します。

    Z: (n_merges x 4) array like [c1, c2, dist, count]
    n_points: 葉の数

    Returns: icoord, dcoord, leaf_order
    """
    # --- 1. ノード情報の準備 ---
    n_nodes = 2 * n_points - 1
    nodes = [{'x': None, 'y': 0.0, 'size': 1, 'left': None, 'right': None} for _ in range(n_points)]

    # Z の各行は c1, c2, dist, count
    for i in range(n_points - 1):
        c1, c2, dist, count = Z[i]
        nodes.append({
            'x': None,
            'y': float(dist),
            'size': int(count),
            'left': int(c1),
            'right': int(c2)
        })

    # --- 2. ソートロジックなしの順序取得 ---
    
    def get_leaf_order_no_sort(node_idx):
        node = nodes[node_idx]
        if node_idx < n_points:
            # リーフノードの場合、自身のIDを返す（再帰の終了条件）
            return [node_idx]
        
        C1_idx, C2_idx = node['left'], node['right']
        
        # Z 行列に記録された C1, C2 の順序をそのまま保持し、回転しない
        order_left = get_leaf_order_no_sort(C1_idx)
        order_right = get_leaf_order_no_sort(C2_idx)
        
        return order_left + order_right

    # --- 3. 座標の計算 ---
    
    def calculate_x_coord(node_idx, leaf_to_x):
        node = nodes[node_idx]
        if node_idx < n_points:
            x_coord = leaf_to_x[node_idx]
            node['x'] = x_coord
            return x_coord
        x_left = calculate_x_coord(node['left'], leaf_to_x)
        x_right = calculate_x_coord(node['right'], leaf_to_x)
        x_coord = (x_left + x_right) / 2.0
        node['x'] = x_coord
        return x_coord
    
    root_node_idx = n_points - 1 + (n_points - 1)
    
    # ソートなしでリーフの順序を取得
    leaf_order = get_leaf_order_no_sort(root_node_idx)
    
    # リーフの順序に基づいてX座標を割り当て
    leaf_to_x = {leaf_idx: 2 * i + 1 for i, leaf_idx in enumerate(leaf_order)}
    calculate_x_coord(root_node_idx, leaf_to_x)

    # --- 4. 座標情報の抽出 ---
    icoord = []
    dcoord = []
    for i in range(n_points - 1):
        P = n_points + i
        C1 = nodes[P]['left']
        C2 = nodes[P]['right']
        y_P = nodes[P]['y']
        y_C1 = nodes[C1]['y']
        y_C2 = nodes[C2]['y']
        x_P = nodes[P]['x']
        x_C1 = nodes[C1]['x']
        x_C2 = nodes[C2]['x']
        
        # デンドログラムの垂直線と水平線を定義
        icoord.append([x_C1, x_C1, x_C2, x_C2])
        dcoord.append([y_C1, y_P, y_P, y_C2])

    return icoord, dcoord, leaf_order

In [9]:

def compute_dendrogram_coords_sorted(Z, n_points, leaf_distances=None, sort_by='size'):
    """
    Linkage Matrixからデンドログラム描画用の座標を計算
    ... (ドキュメンテーションは省略) ...
    """
    # --- 1. ノード情報の準備 (変更なし) ---
    n_nodes = 2 * n_points - 1
    nodes = [{'x': None, 'y': 0.0, 'size': 1, 'left': None, 'right': None} for _ in range(n_points)]

    for i in range(n_points - 1):
        c1, c2, dist, count = Z[i]
        nodes.append({
            'x': None,
            'y': float(dist),
            'size': int(count),
            'left': int(c1),
            'right': int(c2)
        })

    # --- 2. ソートロジックの定義 (変更部分) ---
    
    def get_leaf_order_sorted(node_idx):
        node = nodes[node_idx]
        if node_idx < n_points:
            return [node_idx]
        
        C1_idx, C2_idx = node['left'], node['right']

        # 左右の枝に含まれるリーフノードの順序を再帰的に取得
        order_C1 = get_leaf_order_sorted(C1_idx)
        order_C2 = get_leaf_order_sorted(C2_idx)
        
        # --- 枝の交換決定ロジック ---
        
        # 初期状態: C1が左、C2が右
        current_C1, current_C2 = order_C1, order_C2

        # 1. 第一段階: サイズソート (size) を適用
        # サイズが大きい方を左に配置する
        if nodes[C1_idx]['size'] < nodes[C2_idx]['size']:
            # サイズソート基準で交換が必要
            current_C1, current_C2 = order_C2, order_C1 # 交換を実行 (サイズソートの結果を保持)
        # else: サイズが大きい方を左にするため交換なし
        
        final_order = current_C1 + current_C2 # サイズソート後の暫定順序

        # 2. 第二段階: 類似度ソート (optimal) を適用（sort_by='optimal'が指定された場合）
        if sort_by == 'optimal' and leaf_distances is not None:
            
            # --- ここで、類似度ソートのロジックを適用し、結果を上書き ---
            
            # 枝のサイズを無視し、類似度のみに基づいて交換が必要か判定する
            
            # 回転前: C1の右端と C2の左端 の距離
            dist_normal = leaf_distances[order_C1[-1], order_C2[0]]
            
            # 回転後: C2の右端と C1の左端 の距離
            dist_swapped = leaf_distances[order_C2[-1], order_C1[0]]
            
            # 類似度ソート基準で交換後の方が近ければ、サイズソートの結果を無視して交換
            if dist_swapped < dist_normal:
                final_order = order_C2 + order_C1 # 類似度基準で交換された順序
            else:
                final_order = order_C1 + order_C2 # 類似度基準で交換されない順序

        # サイズソート単独、または類似度ソート単独（指定された場合）の結果を返す
        # 上記のロジックは、sizeソートの結果を類似度ソートが上書きする形になります。
        return final_order
    
    def calculate_x_coord(node_idx, leaf_to_x):
        node = nodes[node_idx]
        if node_idx < n_points:
            x_coord = leaf_to_x[node_idx]
            node['x'] = x_coord
            return x_coord
        x_left = calculate_x_coord(node['left'], leaf_to_x)
        x_right = calculate_x_coord(node['right'], leaf_to_x)
        x_coord = (x_left + x_right) / 2.0
        node['x'] = x_coord
        return x_coord

    # --- 3. デンドログラム座標の計算 (変更なし) ---
    root_node_idx = n_points - 1 + (n_points - 1)
    leaf_order = get_leaf_order_sorted(root_node_idx)
    
    # ... (X座標の計算とicoord, dcoordの構築は既存ロジックをそのまま使用) ...
    # 省略部分は変更なし
    leaf_to_x = {leaf_idx: 2 * i + 1 for i, leaf_idx in enumerate(leaf_order)}
    calculate_x_coord(root_node_idx, leaf_to_x)

    icoord = []
    dcoord = []
    for i in range(n_points - 1):
        P = n_points + i
        C1 = nodes[P]['left']
        C2 = nodes[P]['right']
        y_P = nodes[P]['y']
        y_C1 = nodes[C1]['y']
        y_C2 = nodes[C2]['y']
        x_P = nodes[P]['x']
        x_C1 = nodes[C1]['x']
        x_C2 = nodes[C2]['x']
        icoord.append([x_C1, x_C1, x_C2, x_C2])
        dcoord.append([y_C1, y_P, y_P, y_C2])

    return icoord, dcoord, leaf_order

# 可視化

In [11]:
icoord, dcoord, leaf_order = compute_dendrogram_coords_no_sort(linkage_matrix[:, [0,1,3,4]], n_clusters)
segments = get_dendrogram_segments(icoord, dcoord)
plot_dendrogram_plotly(segments).show()

icoord, dcoord, leaf_order = compute_dendrogram_coords_sorted(linkage_matrix[:, [0,1,3,4]], n_clusters, leaf_distances=cluster_similarity, sort_by='size')
segments = get_dendrogram_segments(icoord, dcoord)
plot_dendrogram_plotly(segments).show()

icoord, dcoord, leaf_order = compute_dendrogram_coords_sorted(linkage_matrix[:, [0,1,3,4]], n_clusters, leaf_distances=cluster_similarity, sort_by='optimal')
segments = get_dendrogram_segments(icoord, dcoord)
plot_dendrogram_plotly(segments).show()

# 評価

In [13]:
import numpy as np

def evaluate_leaf_ordering(leaf_order, leaf_distances):
    """
    デンドログラムの葉の並び順における隣接距離を算出する
    
    Args:
        leaf_order (list): ソートされたリーフIDのリスト
        leaf_distances (np.ndarray): クラスタ間の距離行列 (N x N)
        
    Returns:
        dict: 定量評価結果 (総距離、平均距離、正規化指標)
    """
    # 1. 隣接するペアの距離を抽出
    # leaf_orderが [1, 5, 3] なら、(1,5) と (5,3) の距離を計算
    adjacent_distances = [
        leaf_distances[leaf_order[i], leaf_order[i+1]] 
        for i in range(len(leaf_order) - 1)
    ]
    
    total_dist = sum(adjacent_distances)
    avg_dist = np.mean(adjacent_distances)
    
    # 2. 比較用の指標: ランダムな並びと比較してどの程度改善したか
    # 全組み合わせの平均距離をベースラインとする
    baseline_dist = np.mean(leaf_distances[np.triu_indices(len(leaf_distances), k=1)])
    improvement_ratio = 1.0 - (avg_dist / baseline_dist) if baseline_dist != 0 else 0

    return {
        "total_adjacent_distance": total_dist,    # Σ dist(i, i+1)
        "average_adjacent_distance": avg_dist,   # 平均的な隣接の近さ
        "improvement_over_random": improvement_ratio, # 1に近いほど最適化されている
        "raw_distances": adjacent_distances      # 個別の距離（分布確認用）
    }

# --- 使用例 ---
# icoord, dcoord, leaf_order = compute_dendrogram_coords_sorted(...)
# metrics = evaluate_leaf_ordering(leaf_order, leaf_distances)

# print(f"総隣接距離: {metrics['total_adjacent_distance']:.4f}")
# print(f"ランダム配置に対する改善率: {metrics['improvement_over_random']:.2%}")

In [15]:
# デフォルトの並び順での評価
icoord, dcoord, leaf_order_default = compute_dendrogram_coords_no_sort(linkage_matrix[:, [0,1,3,4]], n_clusters)
metrics_default = evaluate_leaf_ordering(leaf_order_default, cluster_similarity)
print("=== デフォルトの並び順の評価 ===")
print(f"総隣接距離: {metrics_default['total_adjacent_distance']:.4f}")
print(f"ランダム配置に対する改善率: {metrics_default['improvement_over_random']:.2%}")      

# サイズソートの並び順での評価
icoord, dcoord, leaf_order_size = compute_dendrogram_coords_sorted(linkage_matrix[:, [0,1,3,4]], n_clusters, leaf_distances=cluster_similarity, sort_by='size')
metrics_size = evaluate_leaf_ordering(leaf_order_size, cluster_similarity)
print("=== サイズソートの並び順の評価 ===")
print(f"総隣接距離: {metrics_size['total_adjacent_distance']:.4f}")
print(f"ランダム配置に対する改善率: {metrics_size['improvement_over_random']:.2%}")

# 類似度ソートの並び順での評価
icoord, dcoord, leaf_order_optimal = compute_dendrogram_coords_sorted(linkage_matrix[:, [0,1,3,4]], n_clusters, leaf_distances=cluster_similarity, sort_by='optimal')
metrics_optimal = evaluate_leaf_ordering(leaf_order_optimal, cluster_similarity)
print("=== 類似度ソートの並び順の評価 ===")
print(f"総隣接距離: {metrics_optimal['total_adjacent_distance']:.4f}")
print(f"ランダム配置に対する改善率: {metrics_optimal['improvement_over_random']:.2%}")  



=== デフォルトの並び順の評価 ===
総隣接距離: 23181.1708
ランダム配置に対する改善率: 24.56%
=== サイズソートの並び順の評価 ===
総隣接距離: 25701.2466
ランダム配置に対する改善率: 16.36%
=== 類似度ソートの並び順の評価 ===
総隣接距離: 18185.0726
ランダム配置に対する改善率: 40.82%
